In [2]:
import os,sys
sys.path.append('misc/lib/python3.7/site-packages')
#PigmentHunter is added for a file path

from IPython.display import display
import math
import numpy as np
import nglview as nv
import matplotlib.pyplot as plt
import parmed as pmd
import numpy.random as random
import re
import requests
from hubzero.submit.SubmitCommand import SubmitCommand
import time
import hublib.tool as tool
import hublib.ui as ui
import ipywidgets as widgets


from IPython.display import Javascript, display


class ExcStruc:
    def __init__(self):
        self.nres = 0
        self.resnums = []
        self.restypes = []
        self.ham = []
        self.dips = []
        self.cents = []
            
    def reset(self):
        self.nres = 0
        self.resnums = []
        self.restypes = []
        self.ham = []
        self.dips = []
        self.cents = []

class RepList:
    def __init__(self):
        self.component_id = ''
        self.params = []
        self.names = []
        self.reps = list()
        
    def reset(self):
        self.component_id = ''
        self.params = []
        self.names = []
        self.reps = list()
        
    def append(self, nrep):
        self.names.append(nrep.name)
        ptext = {"type": nrep.type, "params": {"color": nrep.color, "sele": nrep.selection, "opacity": str(nrep.opacity)}}
        self.params.append(ptext)
        self.reps.append(nrep)
        
class NGLRep:
    def __init__(self, name, rtype, sel, col, opac):
        self.name = name
        self.type = rtype
        self.selection = sel
        self.color = col
        self.opacity = opac

def check_atoms(RefAtNames, QuerAtNames):
    ismatch = True
    for atnm in RefAtNames:
        if(QuerAtNames.count(atnm)!=1):
            ismatch = False
    return ismatch

def find_porph(pmdstruc, xtruc):
    PORatList = list(['NA', 'NB', 'NC', 'ND'])
    for n in range(0, len(pmdstruc.residues)):
        r = pmdstruc.residues[n]
        nmlist = list()
        for at in r:
            nmlist.append(at.name)
        if(check_atoms(PORatList, nmlist)):
            if(nmlist.count('MG')==1):
                xstruc.resnums.append(n)
                xstruc.nres += 1 
                xstruc.restypes.append('CHL')
                
            else:
                xstruc.resnums.append(n)
                xstruc.nres += 1 
                xstruc.restypes.append('PHO')

    return


def find_dipoles(struc,xtruc):
    ResNums = xtruc.resnums
    DipMat = np.zeros((len(ResNums),3))
    for n in range(0, len(ResNums)):
        r = struc.residues[ResNums[n]]
        for at in r:
            if at.name=='NB':
                NB = struc.coordinates[at.idx]
            if at.name=='ND':
                ND = struc.coordinates[at.idx]
        DipMat[n,:] = NB - ND
    xtruc.dips = DipMat

def find_centers(struc, xtruc):
    ResNums = xtruc.resnums
    CentMat = np.zeros((len(ResNums),3))
    for n in range(0, len(ResNums)):
        r = struc.residues[ResNums[n]]
        cent = 0.0
        for at in r:
            if at.name=='NB':
                cent += 0.5*struc.coordinates[at.idx]
            if at.name=='ND':
                cent += 0.5*struc.coordinates[at.idx]
        CentMat[n,:] = cent
    xtruc.cents = CentMat

def sync_widgets_to_rep(rep):
    seldrop.value = rep.name
    styledrop.value = rep.type
    colordrop.value = rep.color
    opacslide.value = int(rep.opacity*100)
    
def sync_rep_to_widgets():
    global rList
    num = rList.names.index(seldrop.value)
    rep = rList.reps[num]
    rep.type = styledrop.value
    rep.name = seldrop.value
    rep.color = colordrop.value
    rep.opacity = opacslide.value*0.01
    ptext = {"type": rep.type, "params": {"color": rep.color, "sele": rep.selection, "opacity": str(rep.opacity)}}
    rList.params[num] = ptext
    pdbview.set_representations(rList.params)
    

def std_rep(nglview, nvstruc, xstruc):
    global rList
    pdbview.add_trajectory(struc)
    pdbview.clear(0)
    chltxt = ''
    photxt = ''
    for n in range(0, len(xstruc.resnums)):
        if xstruc.restypes[n]=='CHL':
            if len(chltxt)>0:
                chltxt += ' or '
            chltxt += str(xstruc.resnums[n]+1)
        if xstruc.restypes[n]=='PHO':
            if len(photxt)>0:
                photxt += ' or '
            photxt += str(xstruc.resnums[n]+1)
    
    rList.append(NGLRep("Protein", "cartoon", "protein", "grey", 0.2))
    if len(chltxt)>0:
        rList.append(NGLRep("Chlorophyll", "licorice", chltxt, "green", 1.0))
    if len(photxt)>0:
        rList.append(NGLRep("Pheophytin", "licorice", photxt, "blue", 1.0))
    
    pdbview.set_representations(rList.params)
    
    seldrop.options=rList.names
    seldrop.disabled=False
    
    styledrop.disabled=False
    
    colordrop.disabled=False
    opacslide.disabled=False
    sync_widgets_to_rep(rList.reps[0])
    
    for n in range(0, np.shape(xstruc.dips)[0]):
        cent = xstruc.cents[n,:]
        dip = xstruc.dips[n,:]
        v1 = cent - 1.5*dip
        v2 = cent + 1.5*dip
        pdbview.shape.add_arrow(v1.tolist(), v2.tolist(), [1,0,0 ], 1.0 )
        
        
pdbid = widgets.Text(
    value='2DRE',
    placeholder='Type something',
    description='PDB ID:',
    layout = widgets.Layout(width='4cm'),
    disabled=False
)

pdbgo = widgets.Button(
    description='Display',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to display the pdb file',
    icon='' # (FontAwesome names without the `fa-` prefix)
)

pdboutput = widgets.HTML(
    value="",
    placeholder='',
    description='',
)

Output = widgets.Output()

def clear_stage(view):
    view._clear_component_auto_completion()
    if view._trajlist:
        for traj in view._trajlist:
            view._trajlist.remove(traj)
    for component_id in view._ngl_component_ids:
        component_index = view._ngl_component_ids.index(component_id)
        view._ngl_component_ids.remove(component_id)
        view._ngl_component_names.pop(component_index)
        view._remote_call('removeComponent',
            target='Stage',
            args=[component_index,])
    # Shapes appear not to be populated in component_id list.
    # So we have to call seperately removeAllComponents to 
    # get rid of them. 
    view._remote_call('removeAllComponents',
            target='Stage',
            args=[0])
    view._update_component_auto_completion()


def pdbgo_onclick(b):
    global struc
    global estruc
    url = 'http://files.rcsb.org/download/'+pdbid.value+'.pdb'
    r = requests.get(url, allow_redirects=True)
    if(r.status_code==200):
        fname = pdbid.value + '.pdb'
        wfd = open(fname, 'wb')
        wfd.write(r.content)
        wfd.close()
        pdboutput.value = ''
        
        xstruc.reset()
        clear_stage(pdbview)
        rList.reset()
        struc = pmd.load_file(fname)
        find_porph(struc, xstruc)
        find_dipoles(struc, xstruc)
        find_centers(struc, xstruc)
        std_rep(pdbview, struc, xstruc)

    else:
        pdboutput.value = 'Please enter a valide PDB ID code.'

        
xstruc = ExcStruc()
pdbgo.on_click(pdbgo_onclick)
pdbid.on_submit(pdbgo_onclick)
pdbview = nv.NGLWidget()
pdbview._set_size('500px', '500px')
pdbbox = widgets.HBox([pdbid, pdbgo])
rList = RepList()


seldrop = widgets.Dropdown(
    options=rList.names,
    #value='Protein',
    description='Selection:',
    disabled=True,
)


styledrop = widgets.Dropdown(
    options=['cartoon', 'licorice', 'spacefill'],
    description='Style:',
    disabled=True,
)

colordrop = widgets.Dropdown(
    options=['chain', 'red', 'green', 'blue', 'grey'],
    description='Color:',
    disabled=True,
)

opacslide = widgets.IntSlider(
    min=0,
    max=100,
    step=1,
    description='Opacity:',
    disabled=True,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

dispbox = widgets.VBox([seldrop, styledrop, colordrop, opacslide])
viewbox = widgets.HBox([pdbview, dispbox])


def seldrop_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        rep = rList.reps[rList.names.index(seldrop.value)]
        sync_widgets_to_rep(rep)
        
def styledrop_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        num = rList.names.index(seldrop.value)
        rep = rList.reps[num]
        rep.type = styledrop.value
        ptext = {"type": rep.type, "params": {"color": rep.color, "sele": rep.selection, "opacity": str(rep.opacity)}}
        rList.params[num] = ptext
        pdbview.set_representations(rList.params)
        
def colordrop_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        num = rList.names.index(seldrop.value)
        rep = rList.reps[num]
        rep.color = colordrop.value
        ptext = {"type": rep.type, "params": {"color": rep.color, "sele": rep.selection, "opacity": str(rep.opacity)}}
        rList.params[num] = ptext
        pdbview.set_representations(rList.params)
        
def opacslide_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        num = rList.names.index(seldrop.value)
        rep = rList.reps[num]
        rep.opacity = opacslide.value*0.01
        ptext = {"type": rep.type, "params": {"color": rep.color, "sele": rep.selection, "opacity": str(rep.opacity)}}
        rList.params[num] = ptext
        pdbview.set_representations(rList.params)

seldrop.observe(seldrop_on_change)
styledrop.observe(styledrop_on_change)
colordrop.observe(colordrop_on_change)
opacslide.observe(opacslide_on_change)
    
inprogress = False
        
display(pdbbox)
display(pdboutput)
display(viewbox)
struc = pmd.structure

<IPython.core.display.Javascript object>

HTML(value='', placeholder='')

In [179]:
ResAtomNum=[] # this can be removed in next version of code, origonally used to record atom numbers in general
ResAtomName=[] #origonally used to record the atom name in a matrix but a different variant is used further down
ResID=[] #origonally used to record the residue number
N_index=[] #records the nitrogen coordinates
N_id=[] #records the nitrogen residue
N_name=[] #records the nitrogen atom number
O_index=[] #records the oxygen coordinates
O_id=[] #records the oxygen residue
O_name=[] #records the oxygen atom number
C_index=[] #records the carbon coordinates
C_id=[] #records the carbon residue
C_name=[] #records the carbon atom number

for r in struc.residues:
    if "HOH" not in str(r): #enables the ignoring of water residues
        Atoms=[]
        AtomsID=[]
        for at in r:
            if " C  " or "O " or "N " == str(at.name): #finds all carbons, oxygens and nitrogens
                if str(re.match(r"[C,O,N][A-Z]",str(at.name))) =="None": #ignores all other versions of C,O and N that don't exactly match to C, N, or O example being CA
                    Atoms.append(at.number)
                    AtomsID.append(at.name)
                    if "C" ==str(at.name):
                        C_mat=[at.xx,at.xy,at.xz] # records the x,y,z coordinates of each carbon atom
                        C_index.append(C_mat) # records the matrix and creats a n by 3 matrix
                        C_id.append(r.number) #used to record residues but can be removed
                        C_name.append(at.number) # records the number of the atom
                    if "O" ==str(at.name):
                        O_mat=[at.xx,at.xy,at.xz]
                        O_index.append(O_mat)
                        O_id.append(r.number)
                        O_name.append(at.number)
                    if "N" ==str(at.name):
                        N_mat=[at.xx,at.xy,at.xz]
                        N_index.append(N_mat)
                        N_id.append(r.number)
                        N_name.append(at.number)
        ResAtomName.append(AtomsID)
        ResAtomNum.append(Atoms)
        ResID.append(r.number)
        
Amid_bond_mat=[]#matrix that attempts to create a set of amide bonds based on the carbon. Not by residue but based on proximity
for n in range (0,len(C_single)):
    Amid1=[]#this is a temporary matrix Order recorded as C, O, N
    C_atom=C_index[n]
    Amid1.append(C_name[n])
    bondlength_CO=np.sqrt(np.sum(np.square(np.subtract(C_atom, O_index)),1))
    for m in range (0,len(bondlength_CO)):
        if np.greater_equal(1.23*1.15,bondlength_CO[m]) and np.less_equal(1.23*.85,bondlength_CO[m]):
            Amid1.append(O_name[m])
    bondlength_CN=np.sqrt(np.sum(np.square(np.subtract(C_atom, N_index)),1)) #determines the distance between a carbon and all nitrogens
    for m in range (0,len(bondlength_CN)): #this evaluates each point
        if np.greater_equal(1.325*1.15,bondlength_CN[m]) and np.less_equal(1.325*.85,bondlength_CN[m]): #sets a specific range  currently at a 5% tolerance
            Amid1.append(N_name[m])

    Amid_bond_mat.append(Amid1)



In [180]:
print(Amid_bond_mat)

[[3, 4, 7], [9, 10, 14], [16, 17, 28], [30, 31, 40], [42, 43, 44], [46, 47, 51], [53, 54, 59], [61, 62, 70], [72, 73, 77], [79, 80, 86], [88, 89, 98], [100, 101, 106], [108, 109, 110], [112, 113, 117], [119, 120, 128], [130, 131, 134], [136, 137, 138], [140, 141, 147], [149, 150, 153], [155, 156, 160], [162, 163, 166], [168, 169, 178], [180, 181, 186], [188, 189, 193], [195, 196, 197], [199, 200, 206], [208, 209, 217], [219, 220, 224], [226, 227, 228], [230, 231, 236], [238, 239, 248], [250, 251, 252], [254, 255, 266], [268, 269, 274], [276, 277, 281], [283, 284, 286], [288, 289, 290], [292, 293, 298], [300, 301, 304], [306, 307, 309], [311, 312, 317], [319, 320, 324], [326, 327, 333], [335, 336, 340], [342, 343, 351], [353, 354, 356], [358, 359, 365], [367, 368, 373], [375, 376, 384], [386, 387, 393], [395, 396, 401], [403, 404, 410], [412, 413, 417], [419, 420, 425], [427, 428, 435], [437, 438, 441], [443, 444, 452], [454, 455, 466], [468, 469, 471], [473, 474, 479], [481, 482, 487],

In [166]:
print(np.subtract(C_atom,N_single))
#print(len(ResAtomNum))
#print(at.bond_partner)
#help(at)
#print(len(ResAtomName))
#print(ResAtomName)
#print(ResAtomNum)

[-1.47961904e+01 -1.35993083e+01 -1.20417766e+01 -1.09025713e+01
 -9.62376775e+00 -7.42434342e+00 -8.00437611e+00 -7.04257527e+00
 -7.49273368e+00 -8.82036974e+00 -1.01837338e+01 -8.90468696e+00
 -7.43780420e+00 -4.81713711e+00 -1.35788310e+00  1.41932628e+00
  4.92457324e+00  4.77150631e+00  4.50892504e+00  2.62688918e+00
  4.47456591e-01 -4.96097210e-01 -3.51135250e+00 -6.11219529e+00
 -9.33588271e+00 -1.18715017e+01 -1.20144374e+01 -1.23738319e+01
 -1.15458873e+01 -8.30991436e+00 -5.06463090e+00 -2.73104475e+00
 -9.79526452e-01  1.68164212e+00  4.99470970e+00  4.97957621e+00
  4.68841120e+00  2.61090149e+00  1.36386171e+00  1.53950216e+00
  3.22944995e+00  5.57808757e+00  8.52336018e+00  1.00560965e+01
  9.61549439e+00  8.30765735e+00  4.89025631e+00  2.20727103e+00
  6.28086087e-01 -2.33660471e+00 -2.16161590e+00 -3.77562705e+00
 -4.14722261e+00 -5.93520997e+00 -5.02294993e+00 -2.49987133e+00
 -8.30677820e-01  6.47927276e-01  3.49412269e+00  4.96334245e+00
  4.86392200e+00  5.90717